In [1]:
from tkinter import Tk, ttk, Label, Frame, Button, messagebox
import tkinter as tk
import numpy as np
import cv2
import PIL
from PIL import _imaging, Image, ImageTk
import imutils
import tensorflow as tf

import datetime
import time
import threading
import os
import random
import mouse, keyboard
import math
import mediapipe as mp

# Image Processing Class

In [2]:
class ImageProcessing:
    def __init__(self):
        self.using = True
        
    def image_thresholder1(self,stream): #different convolutions on blur
        ycrcb = cv2.cvtColor(stream, cv2.COLOR_BGR2YCrCb)
        blur = cv2.GaussianBlur(ycrcb,(5,5),0) #1,3,5,15,17,51,85,255
        thresh = cv2.inRange(blur, np.array([0, 135, 85]), np.array([255, 180, 135])) #pixelated
        return thresh    
    def image_thresholder2(self,stream): #different convolutions on blur
        ycrcb = cv2.cvtColor(stream, cv2.COLOR_BGR2YCrCb)
        blur = cv2.GaussianBlur(ycrcb,(15,15),0) #1,3,5,15,17,51,85,255
        thresh = cv2.inRange(blur, np.array([0, 135, 85]), np.array([255, 180, 135])) #pixelated
        return thresh
    def image_thresholder3(self,stream):
        gray = cv2.cvtColor(stream, cv2.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY) 
        return thresh
    def image_thresholder4(self,stream):
        gray = cv2.cvtColor(stream, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(5,5),0)
        ret,thresh = cv2.threshold(blur,127,255,cv2.THRESH_BINARY)
        return thresh
    def image_thresholder5(self,stream): ## might be too detailed 
        gray = cv2.cvtColor(stream, cv2.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        return thresh
    def image_thresholder6(self,stream): 
        ycrcb = cv2.cvtColor(stream, cv2.COLOR_BGR2YCrCb)
        gray =  cv2.cvtColor(ycrcb, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(5,5),0) #1,3,5,15,17,51,85,255
        ret,thresh = cv2.threshold(blur,127,255,cv2.THRESH_BINARY)
        return thresh    
    def image_thresholder7(self, stream):
        ycrcb = cv2.cvtColor(stream, cv2.COLOR_BGR2YCrCb)
        gray =  cv2.cvtColor(ycrcb, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(5,5),0) #1,3,5,15,17,51,85,255
        ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        return thresh
    def image_thresholder8(self, stream):
        ycrcb = cv2.cvtColor(stream, cv2.COLOR_BGR2YCrCb)
        gray =  cv2.cvtColor(ycrcb, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(15,15),0) #1,3,5,15,17,51,85,255
        thres_adapt = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 115, 1)
        ret,thresh = cv2.threshold(blur,127,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        return thresh
    
    def image_thresholder9(self, stream):
        #https://github.com/CHEREF-Mehdi/SkinDetection
        img_HSV = cv2.cvtColor(stream, cv2.COLOR_BGR2HSV)
        #skin color range for hsv color space 
        HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17,170,255)) 
        HSV_mask = cv2.morphologyEx(HSV_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

        #converting from gbr to YCbCr color space
        img_YCrCb = cv2.cvtColor(stream, cv2.COLOR_BGR2YCrCb)
        #skin color range for hsv color space 
        YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255,180,135)) 
        YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

        #merge skin detection (YCbCr and hsv)
        global_mask=cv2.bitwise_and(YCrCb_mask,HSV_mask)
        global_mask=cv2.medianBlur(global_mask,3)
        global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, np.ones((4,4), np.uint8))


        #global_result=cv2.bitwise_not(global_mask)
        return global_mask
    
    def image_thresholder0(self, stream):
        # skin detection & compostive filtering if one doesnt work on its own, 
    #might need machine learning or other techniques to adapt skin threshold diff hsv values
        hsv = cv2.cvtColor(stream, cv2.COLOR_BGR2HSV)
    
        h = hsv[:,:,0]
        s = hsv[:,:,1]
        v = hsv[:,:,2]
    
        hsv_split = np.concatenate((h,s,v), axis=1)
    
        ret, min_sat = cv2.threshold(s,40,255, cv2.THRESH_BINARY)
        ret, max_hue = cv2.threshold(h,15,255, cv2.THRESH_BINARY_INV) #inverse
        final = cv2.bitwise_and(min_sat, max_hue)
        
        return final
    



 
    
    def CNN_resizer(self, stream, res_w, res_h):
        resized = imutils.resize(stream, width=res_w, height=res_h)
        return resized
    
    # ----------------------------------------

    def set_roi(self,frame, start_x,start_y, end_x,end_y):
        roi = frame[start_x:end_x, start_y:end_y]
        return roi#, cv2.rectangle(frame, (start_x,start_y),(end_x,end_y), (0,0,255),0)   


    def draw_roi(self, frame, start_x,start_y, end_x,end_y):
        roi = frame[start_x:end_x, start_y:end_y]
        return roi, cv2.rectangle(frame, (start_x,start_y),(end_x,end_y), (0,0,255),0)   

    #------------------------------------------


    def get_fps(self,device):
        fps = device.get(cv2.CAP_PROP_FPS)
        return fps


    def put_text(self,frame, text, pos= (20,20)):
        col = (255,255,255) #rgb
        font = cv2.FONT_HERSHEY_COMPLEX_SMALL
        scale = 1
        thick = 1
        return cv2.putText(frame, str(text), pos, font, scale, col, thick)

    #------------------------------------------ 
    def get_contours(self,stream): 
        contours, hierarchy = cv2.findContours(stream, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        return contours


    def use_contours(self,stream):  
        cnts = self.get_contours(stream) 
        #cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        big_contour = max(cnts, key=cv2.contourArea) #theres a bug with max on an empty list

        # draw filled contour on black background
        filled = np.zeros_like(stream)
        cv2.drawContours(filled, [big_contour], -1, (255), cv2.FILLED)

        # get distance transform
        result = filled.copy()
        result = cv2.distanceTransform(result, distanceType=cv2.DIST_L2, maskSize=3, dstType=cv2.CV_8U)

        return result
    
    def get_hand_centre(self, stream):
        cnts = cv2.findContours(stream, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        
        #if no contour, default to centre
        h, w = stream.shape
        cX = w//2
        cY = h//2
        
        for c in cnts: #can get float division by 0    
            M = cv2.moments(c)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
        return cX, cY

    #------------------------------------------    
    def get_image_centre(self,frame):
        (h, w) = frame.shape[:2]
        h=h//2
        w=w//2
        return h, w

    def draw_guidelines(self,frame, accelregions, h, w):
        cv2.circle(frame,(w,h), accelregions[0], (0,0,255), 2)
        cv2.circle(frame,(w,h), accelregions[1], (0,255,0), 2)
        cv2.circle(frame,(w,h), accelregions[2], (255,0,255), 2)
        cv2.circle(frame,(w,h), accelregions[3], (255,0,0), 2)

# Hand Detector Class 

In [3]:
class handDetector():
    def __init__(self, mode = False, maxHands = 1, detectionCon = 0.5, trackCon = 0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        
    def findHands(self,img, draw = True): #can toggle draw with a button
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS) #draw points and lines
        return img

    def findPosition(self, img, handNo = 0, draw = True):

        lmlist = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmlist.append([id, cx, cy]) #list of all landmark positions
                if draw: #id ==0:
                    cv2.circle(img, (cx, cy), 3, (255, 0, 255), cv2.FILLED)
        return lmlist
    
    
    def findCentre(self, landmarks):
        if len(landmarks) !=0:
            ylow = landmarks[0][1:]  #using fingers isnt good, too much movement, maybe using finger joints
            yhigh = landmarks[9][1:] #12 too high
            xindex = landmarks[5][1:] #4 thumb too wide
            xpinky = landmarks[17][1:] #20 too wide
            
            y = [(ylow[0]+yhigh[0])/2,(ylow[1]+yhigh[1])/2]
            x = [(xindex[0]+xpinky[0])/2,(xindex[1]+xpinky[1])/2]
            c = [int((y[0]+x[0])/2),int((y[1]+x[1])/2)]
            
            return c

Video Thread

# GUI Class

In [4]:
global td
td = []

global ld
ld = []

global cd 
cd = []

In [5]:
class Hand_GUI: 
    
    def __init__(self, master):
        self.master = master
        master.geometry('1400x1000+0+0')
        master.wm_title("HAND GUI")
        self.capture = cv2.VideoCapture(0)
        self.set_res(self.capture, 1280, 720)
        self.ret, self.frame = self.capture.read()
        self.imPRO = ImageProcessing()
        self.detector = handDetector()
        self.menubar = self.menubar(master)
        
        
        #classify and mouse
        #self.model =  tf.keras.models.load_model('\\Models\\Kaggle_BASIC.hdf5') 
        #self.model =  tf.keras.models.load_model('Kaggle_BASIC_reduced_classes_15epoch_280extra_imgs.hdf5') 
        #self.model =  tf.keras.models.load_model('//models//MSc Thesisvgg16_model_basic.hdf5')  cumbersome model
        #self.model =  tf.keras.models.load_model('MSc ThesisNew_None_Save1.hdf5') #
        self.model =  tf.keras.models.load_model('MSc ThesisNewest_None_Save1.hdf5') #*****
        
        
        self.class_names = ['none', 'okay', 'paper', 'rock', 'scissor']
        self.accelregions = [75, 140, 200, 300]
        #create a queue an take modal class of the queue // might see issues here, instead of queue time might be a better
        self.class_queue = [None for _ in range(6)] #10 is a bit of delay
        self.selected_class = self.class_names[0]
        
        
        #starting mouse position https://www.thepythoncode.com/article/control-mouse-python
        self.mp, self.prevmp = mouse.get_position()
    
        #user interface settings
        self.showFPS = False
        
        #event handling selected tabs default
        self.home_frame = True
        self.th_frame = False
        self.ha_frame = False
        self.class_frame = False
        
        self.new_data_frame = False
        self.add_central = False
        self.add_left = False
        self.add_right = False
        
        #event for live minimization
        self.is_Live = False
        
        #toggle between show hands
        self.show_guidelines = True
        
        #get local data
        self.path = os.getcwd()
        
        #default values for threshold tab init
        self.thresh_choice = "image_thresholder8"
        self.thresh_desc = "ycrcb -> gray -> (15,15) guassian blur -> threshold binary with OTSU"
        
        self.threads = []


        self.create_tabs(master)
    
    def set_res(self, cap, x,y):
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, x)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, y)

        
# ------------ TABS -----------------------------------------------------------------------------------------------
    def create_tabs(self,container):
        tabs = ttk.Notebook(container)
        tabs.pack()
        
        go_live_frame = self.go_live_frame(container)
        thresh_frame = self.thresh_frame(container)
        cnn_frame = self.cnn_frame(container)
        hand_frame = self.hand_frame(container)
        new_data_frame = self.add_data_frame(container)
     
        go_live_frame.pack(fill="both", expand=1)
        cnn_frame.pack(fill="both", expand=1)
        hand_frame.pack(fill="both", expand=1)
        thresh_frame.pack(fill="both", expand=1)
        new_data_frame.pack(fill="both", expand=1)
        
        #change gesture -> action tab
        #skin detection/calibration tab
        
        
        tabs.add(go_live_frame, text="Go Live")
        tabs.add(thresh_frame, text="Thresholding")
        tabs.add(hand_frame, text="Mouse Control")
        tabs.add(cnn_frame, text="Classification")
        tabs.add(new_data_frame, text= "Add new data")
        tabs.bind("<<NotebookTabChanged>>", self.tab_change)
        
        
    def tab_change(self, event):
        selected_tab = event.widget.select()
        #print(selected_tab)
        if selected_tab == ".!frame":
            self.home_frame = True
            self.th_frame = False
            self.ha_frame = False
            self.class_frame = False
            self.add_data_frame = False
            self.home()
            #print(0)
        elif selected_tab == ".!frame2":
            self.home_frame = False
            self.th_frame = True
            self.ha_frame = False
            self.class_frame = False
            self.add_data_frame = False
            self.show_thresh()
            #print(1)
        elif selected_tab == ".!frame4":
            self.home_frame = False
            self.th_frame = False
            self.ha_frame = True
            self.class_frame = False
            self.add_data_frame = False
            self.show_hands()
            #print(2)
        elif selected_tab == ".!frame3":
            self.add_data_frame = False
            self.home_frame = False
            self.th_frame = False
            self.ha_frame = False
            self.class_frame = True
            self.show_cnn()
            #print(3)
            
        elif selected_tab ==".!frame5":
            self.add_data_frame = True
            self.home_frame = False
            self.th_frame = False
            self.ha_frame = False
            self.class_frame = False
            self.add_data()
        else:
            print(selected_tab)
        
        
    def go_live_frame(self, container):
        frame = Frame(container, width =1280, height = 720, bg="blue")
        self.imghold = ttk.Label(frame)
        self.imghold.pack()
        btn=Button(frame, text="Go Live", command=self.toggle_live_view)
        btn.pack()
        info = ttk.Label(text="To Exit Please use keyboard: ESC \n to return to application")
        info.pack()
        return frame
    
    def thresh_frame(self, container):
        frame = Frame(container, width =500, height = 500, bg="blue")
        thresh_explanation = ttk.Label(frame, text="This page will change the active thresholding technique across all other pages, use this to experiment and find whichever technique is best")
        thresh_explanation.pack()
        self.thr_hold = ttk.Label(frame)
        self.thr_hold.pack()

        #Drop down menu for changing threshold 
        #https://stackoverflow.com/questions/701802/how-do-i-execute-a-string-containing-python-code-in-python
        thresh_variants = [x for x in dir(self.imPRO) if "thresholder" in x]
        self.thresholder = tk.StringVar()
        self.thresholder.set(thresh_variants[7])
        th_drop = tk.OptionMenu(frame, self.thresholder, *thresh_variants)
        th_drop.pack(padx= 30, pady= 5)
        
        #select threshold to change
        btn=Button(frame, text="Swap Thresholding Technique", command= lambda: self.set_thresh(self.thresholder.get()))
        btn.pack(padx= 30, pady= 5)
        
        #variation description
        self.thr_desc = ttk.Label(frame)
        self.thr_desc.pack(padx= 30, pady= 5,side = tk.RIGHT)
        return frame
    
    def cnn_frame(self, container):
        frame = Frame(container, width=500, height=500, bg="red")
        self.cnn_hold = ttk.Label(frame)
        self.cnn_hold.pack()
        self.hand_pred= ttk.Label(frame)
        self.hand_pred.pack()
        self.hand_conf = ttk.Label(frame)
        self.hand_conf.pack()
        self.det_warning = ttk.Label(frame)
        self.det_warning.pack()
        return frame
    
    def add_data_frame(self, container):
        frame = Frame(container, width=500, height=500, bg="orange")
        self.add_data_img = ttk.Label(frame)
        self.add_data_img.pack()
        leftbtn = Button(frame, text="Left Hand Toggle", command= self.toggle_left) 
        rightbtn = Button(frame, text="Right Hand Toggle", command= self.toggle_right) 
        #drop down menu
        
        self.clicked = tk.StringVar()
        self.clicked.set(self.class_names[0])
        
        drop = tk.OptionMenu(frame, self.clicked, *self.class_names)
        self.add_img = ttk.Label(frame)
        btn=Button(frame, text="Take Photos", command= lambda: self.threader_button(self.clicked.get()))
        
        drop.pack(pady= 5, side = tk.TOP)
        self.add_img.pack(side = tk.BOTTOM)
        leftbtn.pack(padx= 30, pady= 5, side = tk.LEFT)

        btn.pack(padx= 30, pady= 5,side = tk.RIGHT)
        rightbtn.pack(padx= 30, pady= 5,side = tk.LEFT)
        return frame
        
        
    def hand_frame(self, container):
        frame = Frame(container, width=500, height=500, bg="green")
        self.hand_hold= ttk.Label(frame)
        self.hand_hold.pack()
        btn=Button(frame, text="Toggle Guidelines and Skeleton", command= self.toggle_hand_view)
        btn.pack()
        return frame

        
            
                       
        
# -------------- Tab Builders ----------------------------------------------------------------------
    def get_image(self):
        _, frame = self.capture.read()
        frame = cv2.flip(frame,1)
        #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) #looks better on tkinter but scuffs pipeline
        return frame
    
    def normal_image(self):
        if self.home_frame == True:
            frame = self.get_image()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
            image = ImageTk.PhotoImage(image=PIL.Image.fromarray(frame))
            self.imghold.image = image
            self.imghold.configure(image=image)
            self.imghold.after(10, self.normal_image)
            
    def classify(self, image):
        #print("Classify 0", datetime.datetime.now())
        
        image = cv2.resize(image, (240,200), interpolation = cv2.INTER_AREA)
        image = np.expand_dims(image, axis=0)
        image = np.expand_dims(image, axis=-1) #binary images have need to be added back
        #print("Classify 1", datetime.datetime.now())
        #x = datetime.datetime.now()

        #get predictions
        predictions=self.model.predict(image)
        #predictions=self.model(image) #direct seems slower
        #y = datetime.datetime.now()
        #td.append((y-x).total_seconds())
        #print("Classify 2", datetime.datetime.now())
        score = tf.nn.softmax(predictions[0])
        #print("Classify 3", datetime.datetime.now())
        value=np.argmax(predictions)

        #do queue to event handle hold/double/single click
        self.class_queue.pop(0)
        if 100 * np.max(score) < 40:
            self.class_queue.append("Unsure")

        else:
            self.class_queue.append(self.class_names[np.argmax(score)])
        #print("Classify 4", datetime.datetime.now())
        return predictions, score, value
    
    
    def get_hand_pos(self, image):
        hand = self.detector.findHands(image)
        lmlist = self.detector.findPosition(image)
        centre = self.detector.findCentre(lmlist)
        
        #default
        h, w = self.imPRO.get_image_centre(image)
        
        if centre: #if a hand is detected
            hand_cent_X, hand_cent_Y = centre[0], centre[1]
        else:
            hand_cent_X = w #set them in middle if non found
            hand_cent_Y = h
            
        return hand_cent_X, hand_cent_Y
    
    #(240, 200) because dont need below wrist, (190, 210) because thumb sticking out 
    def restrict_roi(self, image, hand_cent_X, hand_cent_Y):
        h, w = image.shape
        #stops going out of bounds
        #stops going out of bounds
        x_warning ="Fine"
        y_warning = "Fine"
        if hand_cent_X <=190 :
            hand_cent_X = 190
            x_warning = "too far left"
        if hand_cent_Y <= 240:
            hand_cent_Y = 240
            y_warning = "too high up"
        if hand_cent_X >= w-211:
            hand_cent_X = w-210
            x_warning = "too far right"
        if hand_cent_Y >= h-201:
            hand_cent_Y = h-200
            y_warning = "too low down"

        #follow hand, probably need to make this more dynamic in the future/ expand size to hand closeness?
        roi = image[hand_cent_Y-240:hand_cent_Y+200, hand_cent_X-190:hand_cent_X+210]
        
        return x_warning, y_warning, roi
    
    
    def can_thread2(self, frame, hX, hY):
        #print("thread", datetime.datetime.now())
        #movement
        h, w = self.imPRO.get_image_centre(frame)
        self.move_mouse(hX, hY, w, h, self.accelregions) 
        self.imPRO.draw_guidelines(frame, self.accelregions, h, w)
        
            
            
            
# ----------- Go Live Tab -------------
    def toggle_live_view(self):
        self.is_Live = not self.is_Live
       

    def home(self):
        #cs = datetime.datetime.now()
        if self.home_frame == True:
            #print("home -1", datetime.datetime.now())
            frame = self.get_image()
            if self.is_Live:
                self.mp, self.prevmp = mouse.get_position()
                try:  
                    if keyboard.is_pressed('esc'):
                        if keyboard.is_press('r'):
                            self.is_Live = not self.is_Live
                except:
                    self.is_Live = not self.is_Live
                
                #print("home 0", datetime.datetime.now())
                #x = datetime.datetime.now()#
                #threshold
                threshed = self.do_selected_thresh(frame.copy())
                #y = datetime.datetime.now()
                #ld.append((y-x).total_seconds())
                #print("home 1", datetime.datetime.now())
                #get hand position
                hand_cent_X, hand_cent_Y = self.get_hand_pos(frame)
                hX, hY = hand_cent_X, hand_cent_Y
            
                t2 = threading.Thread(target=self.can_thread2, args=[frame, hX, hY])
                t2.start()
                #set up acceleration regions
                h, w = self.imPRO.get_image_centre(frame)
                #print("home 2", datetime.datetime.now())
                #stops going out of bounds
                x_warning, y_warning, roi = self.restrict_roi(threshed, hand_cent_X, hand_cent_Y)
                #print("home 3", datetime.datetime.now())
                #classification
                cnn_input = roi.copy()
                predictions, score, value = self.classify(cnn_input)
                #print("home 4", datetime.datetime.now())
                if max(set(self.class_queue), key=self.class_queue.count) == self.class_names[np.argmax(score)]: #threading here
                    self.mp = mouse.get_position() 
                    self.act_on_gesture(self.class_names[np.argmax(score)]) 


                

                #print("home 5", datetime.datetime.now())
                image = ImageTk.PhotoImage(image=PIL.Image.fromarray(frame))
                self.imghold.image = image
                self.imghold.configure(image=image)
                #print("home 6", datetime.datetime.now())
                
       
                
                t2.join()    
            if not self.is_Live:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
                image = ImageTk.PhotoImage(image=PIL.Image.fromarray(frame))
                self.imghold.image = image
                self.imghold.configure(image=image)
            if self.showFPS:
                self.imPRO.put_text(frame, self.imPRO.get_fps(self.capture))
            #ce = datetime.datetime.now()
            #cd.append((ce-cs).total_seconds())
            self.imghold.after(1, self.home) 
            
            
            
# ----------- Threshold Tab ---------------

    def set_thresh(self, choice):
        self.thresh_choice = choice

    def do_selected_thresh(self, frame):
        #maybe have a should max contour show only then,add another layer nested, if button selects show biggest cont
        #exclusively
        if "image_thresholder0" in self.thresh_choice:
            thresh = self.imPRO.image_thresholder0(frame) #2 or 8 optimal
            self.thresh_desc = "LinkedIn Opencv, HSV skin detection bitwise and"    
        if "image_thresholder1" in self.thresh_choice:
            thresh = self.imPRO.image_thresholder1(frame) #2 or 8 optimal
            self.thresh_desc = "ycrcb -> (5,5) guassian blur, in range [0, 135, 85] -> [255, 180, 135]"
        if "image_thresholder2" in self.thresh_choice:
            thresh = self.imPRO.image_thresholder2(frame) 
            self.thresh_desc = "ycrcb -> (15,15) guassian blur, in range [0, 135, 85] -> [255, 180, 135]"
        if "image_thresholder3" in self.thresh_choice:
            thresh = self.imPRO.image_thresholder3(frame)
            self.thresh_desc = "gray -> (15,15) guassian blur, threshold binary"
        if "image_thresholder4" in self.thresh_choice:
            thresh = self.imPRO.image_thresholder4(frame) 
            self.thresh_desc = "gray -> (5,5) guassian blur, threshold binary"
        if "image_thresholder5" in self.thresh_choice:
            thresh = self.imPRO.image_thresholder5(frame) 
            self.thresh_desc = "gray ->  threshold binary  with OTSU"
        if "image_thresholder6" in self.thresh_choice:
            thresh = self.imPRO.image_thresholder6(frame)
            self.thresh_desc = "ycrcb -> gray -> (5,5) guassian blur -> threshold binary "
        if "image_thresholder7" in self.thresh_choice:
            thresh = self.imPRO.image_thresholder7(frame) 
            self.thresh_desc = "ycrcb -> gray -> (5,5) guassian blur -> threshold binary with OTSU"
        if "image_thresholder8" in self.thresh_choice:
            thresh = self.imPRO.image_thresholder8(frame) 
            self.thresh_desc = "ycrcb -> gray -> (15,15) guassian blur -> threshold binary with OTSU"
        if "image_thresholder9" in self.thresh_choice:
            thresh = self.imPRO.image_thresholder9(frame) 
            self.thresh_desc = "https://github.com/CHEREF-Mehdi/SkinDetection"
        return thresh
        
        
    def show_thresh(self):
        if self.th_frame == True:
            frame = self.get_image()
   

            thresh = self.do_selected_thresh(frame)
            self.thr_desc.config(text= self.thresh_desc)
        
            #toggle fps in settings
            if self.showFPS:
                fps = self.imPRO.get_fps(self.capture)
                self.imPRO.put_text(thresh, fps)
            image = ImageTk.PhotoImage(image=PIL.Image.fromarray(thresh))
            self.thr_hold.image = image
            self.thr_hold.configure(image=image)
            self.thr_hold.after(10, self.show_thresh)

# ----------- Hand as mouse Tab ---------------         

    def toggle_hand_view(self):
        self.show_guidelines = not self.show_guidelines

    def show_hands(self):
        if self.ha_frame == True:
            frame = self.get_image()
            hand = frame.copy()
            
            hand_cent_X, hand_cent_Y = self.get_hand_pos(frame)
       
    
            #set up acceleration regions
            h, w = self.imPRO.get_image_centre(frame)
            self.imPRO.draw_guidelines(frame, self.accelregions, h, w)
            
            #draw centre points
            cv2.circle(frame, (hand_cent_X, hand_cent_Y), 3, (255, 0, 255), cv2.FILLED)
            cv2.circle(hand, (hand_cent_X, hand_cent_Y), 3, (255, 0, 255), cv2.FILLED)

            #movement
            self.move_mouse(hand_cent_X, hand_cent_Y, w, h, self.accelregions) #change to focused hand cent, but may rewrite all this
            
            #toggle fps in settings
            if self.showFPS:
                fps = self.imPRO.get_fps(self.capture)
                self.imPRO.put_text(hand, fps)
                self.imPRO.put_text(frame, fps)

            
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
            if self.show_guidelines:
                image = ImageTk.PhotoImage(image=PIL.Image.fromarray(frame))
            if not self.show_guidelines:
                image = ImageTk.PhotoImage(image=PIL.Image.fromarray(hand)) 
            self.hand_hold.image = image
            self.hand_hold.configure(image=image)
            self.hand_hold.after(10, self.show_hands)
            
            
    def move_mouse(self, hand_w, hand_h, window_w, window_h, accelregions):
        acl = 0
        
        x_dir = hand_w - window_w
        y_dir = hand_h - window_h
       
        d =  math.sqrt(abs((abs(x_dir))**2 - (abs(y_dir))**2))
    
    
        #could maybe have user profile to customise these accelerations
        #accelerate the movement depending on distance from centre i.e. if centre no movement, further away = faster
        if d <= accelregions[0]:
            acl = 0
        elif d <= accelregions[1]:
            acl = 0.1
        elif d <= accelregions[2]:
            acl = 0.2
        elif d <= accelregions[3]:
            acl = 0.3
        elif d > accelregions[3]:
            acl = 0.4
        mouse.move(acl*x_dir,acl*y_dir, absolute=False, duration = 0)       
        
       
        
            
# ----------- CNN Tab ---------------

    def show_cnn(self):
        if self.class_frame == True:
            class_names = self.class_names
            frame = self.get_image()
            thresh = frame.copy()
            
            #new hand tracker
            hand = self.detector.findHands(frame)
            lmlist = self.detector.findPosition(frame)
            centre = self.detector.findCentre(lmlist)
            
            #set up acceleration regions
            h, w = self.imPRO.get_image_centre(frame)
            
            if centre: #if a hand is detected
                hand_cent_X, hand_cent_Y = centre[0], centre[1]
            else:
                hand_cent_X = w #set them in middle if non found
                hand_cent_Y = h
                #print("No Hand Found")       
            
            threshed = self.do_selected_thresh(thresh)
            
            h, w, c = frame.shape
            #stops going out of bounds
            x_warning, y_warning, roi = self.restrict_roi(threshed, hand_cent_X, hand_cent_Y)
            
  
            cnn_input = roi.copy()
            predictions, score, value = self.classify(cnn_input)
            
            
            
            if max(set(self.class_queue), key=self.class_queue.count) == class_names[np.argmax(score)]:
                self.mp = mouse.get_position()
                self.act_on_gesture(class_names[np.argmax(score)])
            
            self.hand_pred.config(text="This image most likely belongs to {}".format(class_names[np.argmax(score)]))
            self.hand_conf.config(text="with a {:.2f} confidence, assigning {} from queue".format(100 * np.max(score), max(set(self.class_queue), key=self.class_queue.count)))
            self.det_warning.config(text="X position {}, Y position {}".format(x_warning, y_warning))
            
            image = ImageTk.PhotoImage(image=PIL.Image.fromarray(roi))
            self.cnn_hold.image = image
            self.cnn_hold.configure(image=image)
            self.cnn_hold.after(10, self.show_cnn)
            
    #['none', 'okay', 'paper', 'rock', 'scissor']    
    def act_on_gesture(self, gesture):
        #https://github.com/boppreh/mouse
        
        
        # none or paper
        if gesture == self.class_names[0]:
            mouse.release('left')
            self.class_queue = ["none" for _ in range(6)]
            
            self.prevmp = mouse.get_position()
        
        if gesture == self.class_names[2]:
            mouse.release('left')
            self.class_queue = ["paper" for _ in range(6)]
           
            self.prevmp = mouse.get_position()
            
            
        #okay
        if gesture == self.class_names[1]:
            mouse.release('left')
            mouse.click('middle')
            self.class_queue = ["paper" for _ in range(6)] #buffer for the queue
            
            self.prevmp = mouse.get_position()
            
        #rock 
        if gesture == self.class_names[3]:
            if self.mp == self.prevmp:
                mouse.release('left')
                mouse.press('left')
                self.class_queue = ["paper" for _ in range(6)] #buffer for the queue
                self.prevmp = mouse.get_position()
                
            else:
                mouse.press('left')
                self.class_queue = ["paper" for _ in range(6)] #buffer for the queue
                
                self.prevmp = mouse.get_position()
            
   
        #scissor
        if gesture == self.class_names[4]:
            mouse.release('left')
            mouse.click('right')
            self.class_queue = ["paper" for _ in range(6)] #buffer for the queue
            self.prevmp = mouse.get_position()
            
        
        
        
            
# ----------- Add Data Tab ---------------

    def get_class(self):
        return self.selected_class
        
    def set_class(self, new_class):
        self.selected_class = new_class
        

    def add_data(self):
        if self.add_data_frame == True:
            frame = self.get_image()
            thresh = self.do_selected_thresh(frame)
            #thresh = self.imPRO.image_thresholder8(frame) #2 or 8 optimal
            h, w, c = frame.shape
            if self.add_left == True:
                cv2.rectangle(thresh, (0,0), (486, 406), (255,0,0), 3)
                self.roi = thresh[3:403, 3:483]
            if self.add_right == True:
                cv2.rectangle(thresh, (w-487,0), (w-1, 406), (255,255,0), 3)
                self.roi = thresh[3:403, w-483:w-3]
            image = ImageTk.PhotoImage(image=PIL.Image.fromarray(thresh))
            self.add_data_img.image = image
            self.add_data_img.configure(image=image)
            self.add_data_img.after(10, self.add_data)
            
            #flicks too quickly, either slow it down or cap it. But it does work
            im = self.load_example()
            self.add_img.image = im
            self.add_img.configure(image=im)
            
            
    def toggle_left(self):
        self.add_central = False
        self.add_left = True
        self.add_right = False
        
    def toggle_right(self):
        self.add_central = False
        self.add_left = False
        self.add_right = True
               
    def threader_button(self, class_name):          
        if self.add_left != True and self.add_right != True:
            messagebox.showwarning("Warning", "Please select a position")
            return
        
                        
        #print(threading.active_count())
        t = threading.Thread(target=self.record_section, args = [class_name])
        t.daemon = True # This thread dies when main thread (only non-daemon thread) exits.
        t.start()

        
    def record_section(self, class_name):
        dt = datetime.datetime.now()
        if self.add_left == True:
            hand = "left"
        if self.add_right == True:
            hand = "right"
        i = 0
        #check/create folders
        self.new_image_dir()
        while True:
            current = datetime.datetime.now()
            save_date = current.strftime('%M_%H_%d_%m_%y')
            time_delta = current - dt
            if time_delta.total_seconds() >=10: #have the time (10) as a future custom input
                #print("Done")
                break
            else:
                cv2.imwrite(f'new_images/{class_name}/{hand}hand_{save_date}_{i}.jpg', self.roi)
                #time.sleep(0.1) #1/30 fps have this as a custom input, ask for fps and time
                time.sleep(0.3)
                i+=1
        return #ends a thread
    
    
    def new_image_dir(self):
        image_path = "".join([self.path, "\\", "new_images"])
        if not os.path.exists(image_path):
            os.makedirs(image_path)
        for class_name in self.class_names:
            if not os.path.exists(image_path+"/"+class_name):
                os.makedirs(image_path+"/"+class_name)
                
    def load_example(self):
        class_name = self.clicked.get()       
        image_path = "".join([self.path, "\\", "images", "\\", class_name, "\\"])
        rand = random.choice(os.listdir(image_path))
        im = Image.open(image_path + rand)
        im = im.resize((100,100), Image.ANTIALIAS)
        
        im = ImageTk.PhotoImage(im)
        return im
        

# ------- Menu Bar -----------

    def menubar(self, root):
        menubar = tk.Menu(root)
        
        filemenu = tk.Menu(menubar, tearoff=0)
        filemenu.add_command(label="Help", command = lambda: print("Help"))
        filemenu.add_separator()
        filemenu.add_command(label="Exit", underline=1, command= root.destroy)
        
        uimenu = tk.Menu(menubar, tearoff=0)
        uimenu.add_command(label="UI Settings", command=self.UIsettings)
        uimenu.add_command(label="Load", command= lambda: print("load"))
        uimenu.add_command(label="Save", command= lambda: print("Save"))

        
        usermenu = tk.Menu(menubar, tearoff=0)
        usermenu.add_command(label="Profile Settings", command=self.Usersettings) 
        usermenu.add_command(label="Load", command= lambda: print("load"))
        usermenu.add_command(label="Save", command= lambda: print("save"))
        
        menubar.add_cascade(label="File", menu=filemenu)
        menubar.add_cascade(label="UI", menu=uimenu)
        menubar.add_cascade(label="Profile", menu=usermenu)
        root.config(menu=menubar)
        
        
    def UIsettings(self):
        settings = tk.Toplevel()
        settings.title("Control Panel")
        settings.minsize(300,300)
        Label(settings, text="Customize UI Settings here").grid(row=0, column=0, padx=10, pady=2)
        fpsVar = tk.IntVar()
        tk.Checkbutton(settings, text="Show FPS", variable=fpsVar).grid(row=1,padx=10, pady=2)
        
        Button(settings, text='Change', command= lambda: self.enactUISettings(fpsVar, settings)).grid(row=2, padx=10, pady=2)
        Label(settings, text="Other Ideas: minimise on live [T/F]").grid(row=3, column=0, padx=10, pady=2)
        
        settings.mainloop()
        
    def Usersettings(self):
        settings = tk.Toplevel()
        settings.title("Control Panel")
        settings.minsize(300,300)
        Label(settings, text="Customize User Settings here:\t Acceleration regions: radius and % modifier \t Action to Gesture \t One or two hands \t Which model to use, could have models loaded up with dif gestures \t queue length, how fast you can do successive gestures/gesture sensitivity").grid(row=0, column=1, padx=10, pady=2)
        settings.mainloop()
        
        
        
# ----- Menu Functions ---- 

    def enactUISettings(self, fpsVar, window):
        if fpsVar.get() == 1:
            self.showFPS = True
        elif fpsVar.get() == 0:
            self.showFPS = False
        else:
            print("Error, expected 1 or 0 got other")
        window.destroy()
      
        

        
    

root = Tk()
my_gui = Hand_GUI(root)
root.mainloop()
my_gui.capture.release()
mouse.release(button='left')
mouse.release(button='middle')